<a href="https://colab.research.google.com/github/nathanij/atpPredictor/blob/main/Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [ ]:
!pip install --upgrade tables
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import datetime
import h5py
from google.colab import drive
import keras
from keras.utils import np_utils
drive.mount('/content/drive')

Use processed data

In [20]:
data = pd.read_hdf('/content/drive/MyDrive/tennisData/finished(key=a).h5', key = 'a')

Delete definitely unwanted columns

In [21]:
drops = ['ind', 'tourney_id']
for d in drops:
  data = data.drop(d, axis = 1)
plys = ['p0_', 'p1_']
pDrops = ['id']
for p in plys:
  for d in pDrops:
    data = data.drop(p+d, axis = 1)

Filter out whatever subset you want to train on

In [22]:
data = data[data.tourney_date >= 20080000] #cut out first 3 years of data

Drop Qualifiers

In [23]:
drops = ['tourney_date']
for d in drops:
  data = data.drop(d, axis = 1)
plys = ['p0_', 'p1_']
pDrops = []
for p in plys:
  for d in pDrops:
    data = data.drop(p+d, axis = 1)

Seperate training and validation

In [24]:
data = data.sample(frac=1).reset_index(drop=True) #shuffles rows
training = data.copy()
validation = training[-7000:]
training = training[:-7000]

Found models were unbalanced p0 vs p1 so replicate and flip all training data

In [25]:
oppo = training.copy()
for row in range(len(oppo)):
  for col in oppo:
    if col[:2] == 'p0':
      tmp = oppo.loc[row,col]
      end = col[2:]
      oppo.loc[row, col] = oppo.loc[row, 'p1' + end]
      oppo.loc[row, 'p1' + end] = tmp
  oppo.loc[row, 'winner'] = abs(oppo.loc[row, 'winner'] - 1)

In [26]:
training = pd.concat([training, oppo], ignore_index = True)

In [27]:
training = training.sample(frac=1).reset_index(drop=True) #shuffles rows
np.set_printoptions(precision=3, suppress=True)
y_train = training.pop('winner')
y_train = np_utils.to_categorical(y_train, 2)
x_train = training
x_train = np.asarray(x_train).astype(np.float32)
y_val = validation.pop('winner')
y_val = np_utils.to_categorical(y_val, 2)
x_val = validation
x_val = np.asarray(x_val).astype(np.float32)

Set up checkpoint to save best model while training

In [28]:
acc_filepath = '/content/bestAccModel'
loss_filepath = '/content/bestLossModel'
acc_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=acc_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
loss_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=loss_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
early = tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=50)

In [29]:
model = tf.keras.Sequential([
  layers.Dense(46, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.1)),
  layers.Dense(2048, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.1)),
  layers.Dropout(0.1),                        
  layers.Dense(1024, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.1)),
  layers.Dropout(0.1),
  layers.Dense(512, activation = 'relu', kernel_regularizer=keras.regularizers.l2(l=0.1)),
  layers.Dropout(0.1),
  layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.1)),
  layers.Dropout(0.1),
  layers.Dense(2, activation = 'softmax')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size = 128, epochs = 1000, shuffle=True, callbacks = [acc_callback, loss_callback, early])

In [31]:
accModel = keras.models.load_model('/content/bestAccModel')
lossModel = keras.models.load_model('/content/bestLossModel')

In [32]:
accModel.evaluate(x_val, y_val)

219/219 [==============================] - 2s 8ms/step - loss: 0.6689 - accuracy: 0.6627


[0.668910562992096, 0.6627143025398254]

In [33]:
lossModel.evaluate(x_val, y_val)

219/219 [==============================] - 2s 7ms/step - loss: 0.6487 - accuracy: 0.6606


[0.6487094759941101, 0.6605714559555054]

In [ ]:
accModel.save('r1Acc') #save
!cp -r r1Acc /content/drive/MyDrive/Models
lossModel.save('r1Loss')
!cp -r r1Loss /content/drive/MyDrive/Models